## Lab | Working with APIs

### Instructions

Following the class example, create a function that returns the price, names of origin and arrival airports and the name of the company. Do it for all the flights between two dates that cost the same.

## Working with requests library

In [1]:
import requests
import pandas as pd
import json

In [2]:
#SkyScanner - Rapid API - Keys https://rapidapi.com/skyscanner/api/skyscanner-flight-search
x_rapidapi_key =  "fdafdbf4eamsh05722676738743ap1dd5f9jsn60babf616da5"
shortapikey = "ra66933236979928"

In [3]:
#function to identify city-id 
def city_id(x):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/UK/GBP/en-GB/"

    params = {"query":x}

    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
                          'x-rapidapi-key': x_rapidapi_key}

    # Notice the additional keywords here (headers, params)
    response = requests.get(url, headers = headers, params = params)
    result = response.json()['Places'][0]['PlaceId']
    return result   

In [4]:
city_id('Berlin')

'BER-sky'

In [5]:
#function to find all the flights between two cities on a given date
def flights(city1, city2, date):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/UK/GBP/en-UK/"+city_id(city1)+"/"+city_id(city2)+"/"+date
    #url changes according to the query - that is configured in rapid api
    params = {"inboundpartialdate":date}

    headers = {
        'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
        'x-rapidapi-key': x_rapidapi_key}    

    response = requests.get(url, headers=headers, params=params)

    return response.json()


In [6]:
x = flights('Munich', 'London', "2020-12-12")
x

{'Quotes': [{'QuoteId': 1,
   'MinPrice': 47,
   'Direct': True,
   'OutboundLeg': {'CarrierIds': [1368],
    'OriginId': 68611,
    'DestinationId': 65698,
    'DepartureDate': '2020-12-12T00:00:00'},
   'QuoteDateTime': '2020-11-24T15:38:00'}],
 'Carriers': [{'CarrierId': 1368, 'Name': 'Lufthansa'}],
 'Places': [{'Name': 'London Heathrow',
   'Type': 'Station',
   'PlaceId': 65698,
   'IataCode': 'LHR',
   'SkyscannerCode': 'LHR',
   'CityName': 'London',
   'CityId': 'LOND',
   'CountryName': 'United Kingdom'},
  {'Name': 'Munich',
   'Type': 'Station',
   'PlaceId': 68611,
   'IataCode': 'MUC',
   'SkyscannerCode': 'MUC',
   'CityName': 'Munich',
   'CityId': 'MUNI',
   'CountryName': 'Germany'}],
 'Currencies': [{'Code': 'GBP',
   'Symbol': '£',
   'ThousandsSeparator': ',',
   'DecimalSeparator': '.',
   'SymbolOnLeft': True,
   'SpaceBetweenAmountAndSymbol': False,
   'RoundingCoefficient': 0,
   'DecimalDigits': 2}]}

In [7]:
#function to create a series of dates for the period
def get_dates(start, end):
    return pd.Series(pd.date_range(start, end,freq='d').format()) 
    #.format makes ir object = string, otherwise date

get_dates('1992-01-01','1992-01-05')

0    1992-01-01
1    1992-01-02
2    1992-01-03
3    1992-01-04
4    1992-01-05
dtype: object

In [8]:
#function to identify all the flights between two cities in a period of days

from  time import sleep

def flight_days(origin, destination, start, end):
    dates = get_dates(start, end) #returns panda series
    my_dict = {}
    for date in list(dates):
            try: 
                print(date)
                print("Gathering fligths")
                result = flights(origin, destination, date)
                my_dict[date] = result
                sleep(10)
            except: 
                print("Error from the API")
                sleep(60)
    return my_dict

In [9]:
result = flight_days('Paris','Barcelona','2020-11-26','2020-12-10')

2020-11-26
Gathering fligths
2020-11-27
Gathering fligths
2020-11-28
Gathering fligths
2020-11-29
Gathering fligths
2020-11-30
Gathering fligths
2020-12-01
Gathering fligths
2020-12-02
Gathering fligths
2020-12-03
Gathering fligths
2020-12-04
Gathering fligths
2020-12-05
Gathering fligths
2020-12-06
Gathering fligths
2020-12-07
Gathering fligths
2020-12-08
Gathering fligths
2020-12-09
Gathering fligths
2020-12-10
Gathering fligths


In [10]:
result

{'2020-11-26': {'Quotes': [{'QuoteId': 1,
    'MinPrice': 46,
    'Direct': False,
    'OutboundLeg': {'CarrierIds': [1090],
     'OriginId': 43313,
     'DestinationId': 42414,
     'DepartureDate': '2020-11-26T00:00:00'},
    'QuoteDateTime': '2020-11-23T20:21:00'},
   {'QuoteId': 2,
    'MinPrice': 97,
    'Direct': True,
    'OutboundLeg': {'CarrierIds': [1218],
     'OriginId': 73097,
     'DestinationId': 42414,
     'DepartureDate': '2020-11-26T00:00:00'},
    'QuoteDateTime': '2020-11-24T21:40:00'}],
  'Carriers': [{'CarrierId': 1090, 'Name': 'Ryanair'},
   {'CarrierId': 1218, 'Name': 'Iberia'}],
  'Places': [{'Name': 'Barcelona',
    'Type': 'Station',
    'PlaceId': 42414,
    'IataCode': 'BCN',
    'SkyscannerCode': 'BCN',
    'CityName': 'Barcelona',
    'CityId': 'BARC',
    'CountryName': 'Spain'},
   {'Name': 'Paris Beauvais',
    'Type': 'Station',
    'PlaceId': 43313,
    'IataCode': 'BVA',
    'SkyscannerCode': 'BVA',
    'CityName': 'Paris',
    'CityId': 'PARI',
  

In [11]:
#Overview of json file: 

my_dict = {}

for date in result: # We iterate over the dates
    print(date)
    print("The flights for this date are: ")
    print(result[date]['Quotes'])
    
    my_dict[date] = [] #We create an empty list of prices for the given date.
    
    for i in result[date]['Quotes']: # We iterate over "Quotes" of each day.
        my_dict[date].append(i['MinPrice']) # We fill the date with the prices.
        
    print("The prices for this date are: ",my_dict[date])
    print("The min price for this date is: ",min(my_dict[date])) # We determine the minimum price.
    print()



2020-11-26
The flights for this date are: 
[{'QuoteId': 1, 'MinPrice': 46, 'Direct': False, 'OutboundLeg': {'CarrierIds': [1090], 'OriginId': 43313, 'DestinationId': 42414, 'DepartureDate': '2020-11-26T00:00:00'}, 'QuoteDateTime': '2020-11-23T20:21:00'}, {'QuoteId': 2, 'MinPrice': 97, 'Direct': True, 'OutboundLeg': {'CarrierIds': [1218], 'OriginId': 73097, 'DestinationId': 42414, 'DepartureDate': '2020-11-26T00:00:00'}, 'QuoteDateTime': '2020-11-24T21:40:00'}]
The prices for this date are:  [46, 97]
The min price for this date is:  46

2020-11-27
The flights for this date are: 
[{'QuoteId': 1, 'MinPrice': 56, 'Direct': False, 'OutboundLeg': {'CarrierIds': [1816], 'OriginId': 73097, 'DestinationId': 42414, 'DepartureDate': '2020-11-27T00:00:00'}, 'QuoteDateTime': '2020-11-24T14:30:00'}, {'QuoteId': 2, 'MinPrice': 103, 'Direct': True, 'OutboundLeg': {'CarrierIds': [1865], 'OriginId': 73097, 'DestinationId': 42414, 'DepartureDate': '2020-11-27T00:00:00'}, 'QuoteDateTime': '2020-11-24T14:3

ALternative following the class, icluding resolution for repeated prices on multiple dates: 

In [12]:
def lowest_price(origin, destination, start, end):
    
    flights = flight_days(origin, destination, start, end)
    #dictionary: date : min price 
    prices = {date:min([quote["MinPrice"] for quote in flights[date]["Quotes"]]) for date in flights for quote in flights[date]["Quotes"]}
    
    #looping through prices to find dates that have the min price value
    min_prices_dates = [date for date, price in prices.items() if price == min(prices.values())]
    
    return min_prices_dates

In [13]:
lowest_price('Paris','Barcelona','2020-11-26','2020-12-10')

2020-11-26
Gathering fligths
2020-11-27
Gathering fligths
2020-11-28
Gathering fligths
2020-11-29
Gathering fligths
2020-11-30
Gathering fligths
2020-12-01
Gathering fligths
2020-12-02
Gathering fligths
2020-12-03
Gathering fligths
2020-12-04
Gathering fligths
2020-12-05
Gathering fligths
2020-12-06
Gathering fligths
2020-12-07
Gathering fligths
2020-12-08
Gathering fligths
2020-12-09
Gathering fligths
2020-12-10
Gathering fligths


['2020-11-29', '2020-12-02', '2020-12-06', '2020-12-09']